# Reclassify user/contact organizations

In [32]:
# API settings
api_url = '/services/data/v43.0/sobjects'
object_id = 'Contact'
external_id = 'nanoHUB_user_ID__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"shit9927289sSYTkLiGvyK2UrazHFmjFUGU",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",


    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [33]:
import sys
#sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Obtain user org classifications from Salesforce

In [40]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# get Salesforce ID for organizations
contact_org_df = db_s.query_data('SELECT Id, nanoHUB_user_ID__c, Email, Organization_composite__c,\
Organization_email_derived__c, Organization_Conflict__c, \
Organization__c FROM Contact Order by nanoHUB_user_ID__c DESC limit 5000')

#Organization_citation_derived__c

# display
contact_org_df.head(3)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000PVFV3AAP
{"id":"7505w00000PVFV3AAP","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-01T03:43:26.000+0000","systemModstamp":"2020-09-01T03:43:26.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000PVFV3AAP","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2020-09-01T03:43:26.000+0000","systemModstamp":"2020-09-01T03:43:32.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":5000,"retries":0,"totalProcessingTime":3483}
[Success] Bulk job completed successfully.


,Email,Id,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,jack@test.com,0035w000033yK8wAAE,False,NaN,NaN,NaN,NaN
1,asdf1234@invalidaddress5.org,0035w00003DA2NpAAL,False,NaN,NaN,NaN,NaN
2,defnitelynotanemail7193@purdue.edu,0035w00003DA2MmAAL,False,NaN,NaN,NaN,NaN


In [ ]:
# pull all the organizations from SF to get their unique tokens
all_org_df = db_s.query_data('SELECT Id, Name, Domain__c FROM Organization__c')

display(all_org_df.head(2))
print(all_org_df.shape)

In [ ]:
## extract the domain from contact_org_df
user_domains = contact_org_df['Email'].fillna(' ').apply(lambda x: x.split('@')[-1])
display(user_domains.head(2))

In [35]:
## if contact domain is a commercial email domain, then find the index and empty out the entry 
import commercial_domains as cdomains
user_domains = user_domains.apply(lambda x: ' ' if x in cdomains.domains else x)
display(user_domains.head(2))

0               
1    nanohub.org
Name: Email, dtype: object

In [36]:
## finding indexes in contact_org_df and zero out Organization_email_derived__c
indices = [i for i, x in enumerate(user_domains.to_list()) if x == ' ']

contact_org_df['Organization_email_derived__c'][indices] = ' '

## list comprehension method
display(contact_org_df.head(5))


C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Email,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,nkissebe@gmail.com,False,NaN,NaN,,998.0
1,support@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1683.0
2,gridstat@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1684.0
3,ncn@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1685.0
4,apps@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1686.0


In [18]:
## match/find all_org_df ID values
nh_orgs = contact_org_df['Organization__c'].fillna(' ').apply(lambda x: x.lower())
sf_org_strings = all_org_df['Name'].fillna(' ').apply(lambda x: x.lower()).to_list()
sf_org_ids = all_org_df['Id'].fillna(' ')

In [19]:
# continued from above snippet
indices = [i for i, x in enumerate(nh_orgs.to_list()) if x != ' ']

#check if organization already has a SF entry
for i in indices:
    if nh_orgs[i] in sf_org_strings: #if exact match
        t_index = sf_org_strings.index(nh_orgs[i]) #pull exact id
        nh_orgs[i] = sf_org_ids[t_index]
    else:
        nh_orgs[i] = ' '
        
#    else:
#        #calculate the damerau levenshtein distance
#        min_dl_dist,org_index = dl_dist(nh_orgs[i],sf_org_strings)
#        if min_dl_dist <= 4: #fit the org to the org_index 
#            t_index = org_index
#            nh_orgs[i] = sf_org_ids[t_index]
        ## create new entry is involved... create a new DF, populate with new org entries, send to SF
        ## pull SF assigned new org ID back, then repopulate the contacts SF DF with new org ID only for those new orgs 
        #else: #create new entry
        #    t_index = org_index

In [20]:
display(nh_orgs.head(3))

0                      
1    a0r5w00000V42cCAAR
2    a0r5w00000V42cCAAR
Name: Organization__c, dtype: object

In [21]:
## Compare the SF indexes of NH determined organization against that of Organization_email_derived__c
email_orgs = contact_org_df['Organization_email_derived__c']

for i,j in enumerate(nh_orgs.to_list()):
    if j != ' ' and email_orgs[i] == ' ':
        # nanohub org overwrites all other org flags
        contact_org_df['Organization_composite__c'][i] = j
    elif j != ' ' and j != email_orgs[i]:
        contact_org_df['Organization_Conflict__c'][i] = True
        contact_org_df['Organization_composite__c'][i] = j
    elif j != ' ' and j == email_orgs[i]:
        contact_org_df['Organization_Conflict__c'][i] = False
        
display(contact_org_df.head(4))
display(contact_org_df.tail(4))

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\henry\

,Email,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,nkissebe@gmail.com,False,NaN,NaN,,998.0
1,support@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1683.0
2,gridstat@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1684.0
3,ncn@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1685.0


,Email,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
224592,s3541448@student.rmit.edu.au,False,NaN,NaN,NaN,298409.0
224593,NaN,False,NaN,NaN,,298410.0
224594,samiul.engr@ieee.org,False,NaN,NaN,NaN,298412.0
224595,apatnaik@cvrce.edu.in,False,NaN,NaN,a0r5w00000VNV6CAAX,298382.0


In [22]:
contact_org_df = contact_org_df.drop(columns='Id')

KeyError: "['Id'] not found in axis"

In [23]:
display(contact_org_df.head(2))

,Email,Organization_Conflict__c,Organization__c,Organization_composite__c,Organization_email_derived__c,nanoHUB_user_ID__c
0,nkissebe@gmail.com,False,NaN,NaN,,998.0
1,support@nanohub.org,False,purdue university,a0r5w00000V42cCAAR,a0r5w00000V42cCAAR,1683.0


## To Salesforce Sales Cloud CRM

In [24]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [25]:
# send data to Salesforce
db_s.send_data(contact_org_df)

[Success] Bulk job creation successful. Job ID = 7505w00000PVFGIAA5
hello
[Success] CSV upload successful. Job ID = 7505w00000PVFGIAA5
[Success] Closing job successful. Job ID = 7505w00000PVFGIAA5


In [31]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000PVFGIAA5',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-09-01T03:26:47.000+0000',
 'systemModstamp': '2020-09-01T03:35:19.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 224596,
 'numberRecordsFailed': 37,
 'retries': 0,
 'totalProcessingTime': 2169343,
 'apiActiveProcessingTime': 1976043,
 'apexProcessingTime': 20746392}

In [27]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
